In [1]:
import random

# StringIO behaves like a file object 
from io import StringIO 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
from sklearn.datasets import load_boston
from sklearn import linear_model
from sklearn.metrics import r2_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
import statsmodels.formula.api as smf

import copy


from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


%matplotlib inline 

In [2]:
class CqkProblem:
    def __init__(self, r, n, d, a, b, low, up):
        self.n = n
        self.r = r
        self.d = list(d)
        self.a = list(a)
        self.b = list(b)
        self.low = list(low)
        self.up = list(up)

In [3]:
def generate_cqk_problem(n):
    d = []
    low = []
    up = []
    b = []
    a = []
    temp = 0
    lb = 0.0
    ub = 0.0
    lower = 10
    upper = 25
    r = 0

    for i in range(n):
        
        b.append(10 + 14*random.random())
        low.append(1 + 14*random.random())
        up.append(1 + 14*random.random())
        if low[i] > up[i]:
            temp = low[i]
            low[i] = up[i]
            up[i] = temp
        
        lb = lb + b[i]*low[i];
        ub = ub + b[i]*up[i];
        
        #Uncorrelated
        d.append(random.randint(10,25))
        a.append(random.randint(10,25))
        
    r = lb + (ub - lb)*0.7;
    
    return CqkProblem( r, n, d, a, b, low, up)

In [4]:
def initial_lambda(p, lamb):
    s0=0.0
    q0=0.0
    slopes = []
    for i in range(p.n):
        slopes.append((p.b[i]/p.d[i])*p.b[i])
        s0 = s0 + (p.a[i] * p.b[i]) / p.d[i]
        q0 = q0 + (p.b[i] * p.b[i]) / p.d[i]
    lamb = (p.r-s0)/q0
    return lamb, slopes

In [5]:
def phi_lambda(p,lamb,phi,deriv,slopes,r):
    deriv = 0.0
    phi = r * -1
    x = []
    
    for i in range(p.n):
        
        x.append( (p.b[i] * lamb + p.a[i])/p.d[i])

        if x[i] < p.low[i]:
            x[i] = p.low[i]
        elif x[i] > p.up[i]:
            x[i] = p.up[i]
        else:
            deriv = deriv + slopes[i];
        phi = phi + p.b[i] * x[i];
    return deriv, phi, x

In [16]:
MAX_IT = 10
INFINITO_NEGATIVO = -999999999;
INFINITO_POSITIVO = 999999999;

def newton(p):
    lambs = [] 
    derivs = []
    phis = []
    phi = 0
    lamb = 0
    alfa = INFINITO_NEGATIVO;
    beta = INFINITO_POSITIVO;
    phi_alfa = 0.0;
    phi_beta = 0.0;
    deriv = 0
    x = []
    r = p.r
    
    lamb, slopes = initial_lambda(p,lamb)
    deriv, phi, x = phi_lambda(p,lamb,phi,deriv,slopes,r)
    lambs.append(lamb)
    derivs.append(deriv)
    phis.append(phi)
    
    it = 1
    
    while phi != 0.0 and it <= MAX_IT:
        if phi > 0:
#             print("positivo")
            beta = lamb
            lambda_n = 0.0
            if deriv > 0.0:
                
                lambda_n = lamb - (phi/deriv)
                if abs(lambda_n - lamb) <= 0.00000000001:
                    phi = 0.0
                    break
                if lambda_n > alfa:
                    lamb = lambda_n
                else:
#                     print("aqui")
                    phi_beta = phi;
#                     lamb = secant(p,x,alfa,beta,phi_alfa,phi_beta,r);
#             if deriv == 0.0:
#                 lamb = breakpoint_to_the_left(p,lamb);
#                 if lamb <= INFINITO_NEGATIVO or lamb >= INFINITO_POSITIVO:
#                     break
                
        else:
            if it == 1:
                negativo = True
#             print("negativo")
            alfa = lamb;
            lambda_n = 0.0;

            if deriv > 0.0:
                lambda_n = lamb - (phi/deriv)
                if abs(lambda_n - lamb) <= 0.00000000001:
                    phi = 0.0
                    break
                
                if lambda_n < beta:
                    lamb = lambda_n
                else:
#                     print("aqui")
                    phi_alfa = phi;
#                     lamb = secant(p,x,alfa,beta,phi_alfa,phi_beta,r);
#             if deriv == 0.0:
#                 lamb = breakpoint_to_the_right(p,lamb)
#                 if lamb <= INFINITO_NEGATIVO or lamb >= INFINITO_POSITIVO:
#                     break
        
        
        deriv, phi, x = phi_lambda(p,lamb,phi,deriv,slopes,r)
        it = it + 1
        lambs.append(lamb)
        derivs.append(deriv)
        phis.append(phi)
        
    if phi == 0.0:
        return it,lambs,derivs,phis
    elif alfa == beta:
        return -1,lambs,derivs,phis
    else:
        return -2,lambs,derivs,phis

In [102]:
def phi_lambda_two(p,lamb1,lamb2,lamb3,r):
    
    slopes = []
    
    deriv1 = 0.0
    deriv2 = 0.0
    deriv3 = 0.0
    phi1 = r * -1
    phi2 = r * -1
    phi3 = r * -1
    x1 = []
    x2 = []
    x3 = []
    
    for i in range(p.n):
        
        slopes.append((p.b[i]/p.d[i])*p.b[i])
        
        x1.append( (p.b[i] * lamb1 + p.a[i])/p.d[i])

        if x1[i] < p.low[i]:
            x1[i] = p.low[i]
        elif x1[i] > p.up[i]:
            x1[i] = p.up[i]
        else:
            deriv1 = deriv1 + slopes[i];
        phi1 = phi1 + p.b[i] * x1[i];
        
        ########
        
        x2.append( (p.b[i] * lamb2 + p.a[i])/p.d[i])

        if x2[i] < p.low[i]:
            x2[i] = p.low[i]
        elif x2[i] > p.up[i]:
            x2[i] = p.up[i]
        else:
            deriv2 = deriv2 + slopes[i];
        phi2 = phi2 + p.b[i] * x2[i];
        
        #########
        
        x3.append( (p.b[i] * lamb3 + p.a[i])/p.d[i])

        if x3[i] < p.low[i]:
            x3[i] = p.low[i]
        elif x3[i] > p.up[i]:
            x3[i] = p.up[i]
        else:
            deriv3 = deriv3 + slopes[i];
        phi3 = phi3 + p.b[i] * x3[i];

        
        
    return deriv1, phi1, deriv2, phi2,deriv3, phi3,slopes

In [94]:
lista = []
for i in range(100):
    n = 1000
    p = generate_cqk_problem(n)
    it,lambs, derivs,phis = newton(p)
    soma_a = 0
    soma_b = 0
    soma_low = 0
    soma_d = 0
    soma_up = 0
    for i in range(n):
        soma_a += p.a[i]
        soma_b += p.b[i]
        soma_low += p.low[i]
        soma_d += p.d[i]
        soma_up += p.up[i]
    
    soma_a = soma_a/n
    soma_b = soma_b/n
    soma_low = soma_low/n
    soma_d = soma_d/n
    soma_up = soma_up/n
    r = p.r/n
    deriv = derivs[0]/n
#     l_rs = [soma_a, soma_b, soma_low, soma_up, soma_d, p.r, lambs[0], lambs[-1]]
    if it > 3 and it < 6:
        l_rs = [soma_a, soma_b, soma_d, p.r,lambs[0],lambs[1],lambs[2],lambs[-1],phis[0],phis[1],phis[2],phis[-1],derivs[0],derivs[1],derivs[2],derivs[-1]]
        lista.append(l_rs)
    if len(lista) == 100:
        break

In [95]:
np.savetxt('instance_test.txt', lista, delimiter = ' ',newline='\n', fmt="%f")

In [96]:
c = ''
with open("instance_test.txt", "r") as fd:
    c = StringIO(fd.read())

In [97]:
d = c.read()
c = StringIO(d) 
d = np.loadtxt(c) 
feature_names = ['a','b','d','r','lamb1','lamb2','lamb3','lamb4','phi1','phi2','phi3','phi4','deriv1','deriv2','deriv3','deriv4']

In [98]:
knapsack = {"data":d, "feature_names": feature_names}
dataset = pd.DataFrame(knapsack['data'], columns = knapsack['feature_names'])

In [99]:
dataset

,a,b,d,r,lamb1,lamb2,lamb3,lamb4,phi1,phi2,phi3,phi4,deriv1,deriv2,deriv3,deriv4
0,17.171,16.860214,17.595,150915.749806,7.181860,8.651570,8.850276,8.852892,-10078.047500,-1090.514676,-13.974296,0.0,6857.169916,5488.076699,5341.744535,5341.744535
1,17.379,16.985053,17.284,152539.288882,7.013752,8.765374,8.982715,8.987259,-11868.347692,-1179.974079,-23.969490,-0.0,6775.629783,5429.143411,5275.512816,5275.512816
2,17.602,16.863720,17.281,151903.080179,7.085006,8.588995,8.790675,8.794981,-10929.890497,-1210.215091,-24.732906,0.0,7267.265973,6000.674582,5750.348144,5740.234420
3,17.491,17.121839,17.468,152722.936637,7.008237,8.603438,8.720962,8.723057,-11045.733627,-709.612675,-12.140926,-0.0,6924.355685,6038.026627,5793.753741,5793.753741
4,17.392,16.841670,17.267,152175.201249,7.180773,8.565430,8.750129,8.755843,-8753.495664,-950.474994,-27.561701,0.0,6321.776008,5146.072427,4847.728210,4818.989061
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,17.253,17.170776,17.501,152952.566426,7.048167,8.674297,8.875446,8.879373,-11173.744752,-1102.055876,-20.615969,-0.0,6871.374258,5478.793652,5250.577683,5250.577683
96,17.534,16.829177,17.454,147968.365339,7.019657,8.603897,8.820988,8.825279,-10288.364235,-1146.992002,-21.752585,0.0,6494.194954,5283.469738,5069.264560,5069.264560
97,17.617,17.139286,17.476,151521.920266,6.983425,8.454123,8.655401,8.659514,-10332.755551,-1100.961861,-21.670094,-0.0,7025.746778,5469.853019,5269.790784,5269.790784
98,17.780,16.995716,17.581,153358.161437,7.165536,8.835902,9.103874,9.106935,-10943.159449,-1390.339316,-15.379165,-0.0,6551.354177,5188.372251,5023.897246,5023.897246


In [76]:
#50000
print(sum(dataset['phi1']))
print(sum(dataset['phi2']))
print(sum(dataset['phi3']))
print(sum(dataset['phi4']))

-53226975.924742006
-5301572.509874999
-83281.93797299998
0.0


In [77]:
#50000
print(sum(dataset['deriv1']))
print(sum(dataset['deriv2']))
print(sum(dataset['deriv3']))
print(sum(dataset['deriv4']))

34120731.32111998
27300584.508789994
26447074.129830994
26433854.89548497


In [92]:
#10000
print(sum(dataset['phi1']))
print(sum(dataset['phi2']))
print(sum(dataset['phi3']))
print(sum(dataset['phi4']))

-10734788.657182
-1056271.273601
-15803.388997000002
0.0


In [93]:
#10000
print(sum(dataset['deriv1']))
print(sum(dataset['deriv2']))
print(sum(dataset['deriv3']))
print(sum(dataset['deriv4']))

6872383.803617
5501193.044472999
5337831.986663001
5334868.089718002


In [100]:
#1000
print(sum(dataset['phi1']))
print(sum(dataset['phi2']))
print(sum(dataset['phi3']))
print(sum(dataset['phi4']))


-1080211.8570049996
-105745.20473799999
-1535.0582400000003
0.0


In [101]:
#1000
print(sum(dataset['deriv1']))
print(sum(dataset['deriv2']))
print(sum(dataset['deriv3']))
print(sum(dataset['deriv4']))

680858.8428869997
544290.7695219999
528618.3067040003
528331.3187980002


In [129]:
n = 10000


derivs1 = []
derivs2 = []
derivs3 = []
phis1 = []
phis2 = []
phis3 = []
for i in range(10):
    
    pp = generate_cqk_problem(n)
    it,lambs, derivs,phis = newton(pp)

    deriv1, phi1, deriv2, phi2,deriv3, phi3,slopess= phi_lambda_two(pp,5,7,9,pp.r)
    derivs1.append(deriv1)
    derivs2.append(deriv2)
    derivs3.append(deriv3)
    phis1.append(phi1)
    phis2.append(phi2)
    phis3.append(phi3)


In [110]:
#1000
print(sum(derivs1), sum(phis1))
print(sum(derivs2), sum(phis2))
print(sum(derivs3), sum(phis3))

81597.1469189085 -265041.7236368234
69009.42401158909 -112574.28122833117
51682.31774147703 8751.669272192006


In [130]:
#10000
print(sum(derivs1), sum(phis1))
print(sum(derivs2), sum(phis2))
print(sum(derivs3), sum(phis3))

814937.0880056664 -2666381.2489072443
692577.7764757375 -1137897.6579117244
518361.62566546147 72749.58249889225


In [116]:
#100000
print(sum(derivs1), sum(phis1))
print(sum(derivs2), sum(phis2))
print(sum(derivs3), sum(phis3))

8194024.724736146 -26756279.80947032
6978349.096834154 -11364651.427830026
5214547.425830053 834804.4003592272
